In [2]:
import csv
import numpy as np
import pandas
import math
import random

In [35]:
#cálculo del valor c
#es un valor que depende del número de periodos necesarios
#para que RD, es decir, la desviacón, se vuelva poco confiable
def Cvalue(t):
    return math.sqrt((350**2 - 50**2)/t)

#RDold es el antiguo Rd
#c es el valor de c que hemos asignado dado Cvalue
#t es el número de etapas que lleva inactivo el jugador
def newRD(RDold, c, t):
    val = min(math.sqrt(RDold**2 + c**2*t), 350)
    if val < 20: 
        val = 20
    return val

#esto me lo saco un poco de los cojones pero
#es que es parte de lo que tenemos que mirar
def getPlayersForMatch(playerID, df):
    playerPoints = df['points'][playerID]
    playerRD = df['deviation'][playerID]
    return df.loc[(df['points'] >= (playerPoints - playerRD)) & (df['points'] <= (playerPoints + playerRD))]
    
def generateGames(player, rivals):

    playerPoints = df['points'][player]
    playerDeviation = df['deviation'][player]
    results = []
    numMatches = len(rivals)
    for i in range(0,numMatches):
        result = prediction(playerDeviation, rivals['deviation'].iloc[i], 
                        playerPoints, rivals['points'].iloc[i])
        match = random.random()
        if(match < result):
            results.insert(i, 1)
        else:
            results.insert(i, 0) 
    return results

def g(RD):
    q = 0.0057565
    return 1/(math.sqrt((1 + 3*(q**2)*(RD**2))/math.pi**2))
    
def prediction(RD1, RD2, r1, r2):
    elevateG = -g(math.sqrt(RD1**2 + RD2**2))*(r1-r2)/400
    return 1/(1+10**elevateG)

def E(rivalD, playerPoints, rivalPoints):
    elevateG = -g(rivalD)*(playerPoints-rivalPoints)/400
    return 1/(1+10**elevateG)

def calculateD(player, rivals):
    q = 0.0057565
    predictV = np.vectorize(E)
    gV = np.vectorize(g)
    
    return (1/(q**2 * (np.sum(gV(rivals['deviation'])**2 *
                         predictV(rivals['deviation'],
                                 player['points'],
                                 rivals['points']) * 
                         (1-predictV(rivals['deviation'],
                                 player['points'],
                                 rivals['points']))))))
        
def getRD(RD, d):
    return math.sqrt(1/((1/(RD**2))+(1/(d))))

def newPoints(player, rivals, results):
    q = 0.0057565
    gV = np.vectorize(g)
    predictV = np.vectorize(E)
    
    sum = np.sum(gV(rivals['deviation']) * (results - predictV(rivals['deviation'],
                                                                  player['points'],
                                                                  rivals['points'])))
    d = calculateD(player, rivals)
    newRD = getRD(player['deviation'], d)
    return [player['points'] + (q/(1/(newRD**2) + 1/d)) * sum, newRD]
    

In [43]:
df = pandas.read_csv('players_file.csv')
id = 450
player = df.loc[id]
findings = getPlayersForMatch(id, df)
rivals = findings.sample(n=20)
print(rivals)

results = generateGames(id, rivals)        
print(results)

#deviations = np.array(rivals['deviation'])
#points = np.array(rivals['points'])
#print(deviations)
#c = np.vectorize(prediction)
#f = np.sum(c(df.loc[id]['deviation'],
#                     deviations,
#                     df.loc[id]['points'],
#                     points))

#print(calculateD(df.loc[id], rivals))
print(player)
values = newPoints(player, rivals, results)
print(values)
#c = Cvalue(30)
#print(newRD(player['deviation'],c,0))

      id       points  deviation
980  980  1317.012086  38.740408
688  688  1309.690166  41.654151
225  225  1339.235529  47.766908
473  473  1237.660496  45.372267
873  873  1244.267329  50.464601
976  976  1321.053075  39.724626
99    99  1324.099832  64.147439
324  324  1313.644505  44.016115
965  965  1332.658537  36.024748
909  909  1276.010577  22.026667
218  218  1250.571253  52.999443
483  483  1309.092328  60.334047
944  944  1291.425494  48.505079
446  446  1211.482838  43.855218
385  385  1286.684709  43.816368
837  837  1282.197203  35.839520
70    70  1340.293097  60.835136
418  418  1257.214881  55.138821
241  241  1299.508717  56.440377
979  979  1311.543310  58.998101
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0]
id            450.000000
points       1277.595360
deviation      66.723214
Name: 450, dtype: float64
[1272.7369008313556, 26.341233880173064]
